In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl
from itertools import islice

run_id = 359
qid = 1603

In [2]:
from openpyxl import Workbook, load_workbook

In [3]:
try:
    wb = load_workbook('../data/bibliometric nodes.xlsx')
    ws = wb['useme']
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, index=idx, columns=cols)
except:
    pass

In [4]:
df = pd.read_csv('../data/cities_bib_coupling_nodes.csv')

In [5]:
df['id'] = df.index
df.head()

,Id,Label,timeset,description,url,cluster,weight<links>,weight<total_link_strength>,weight<citations>,weight<norm._citations>,score<pub._year>,score<citations>,score<norm._citations>,id
0,1,echenique (2013),NaN,"<table><tr><td>Authors:</td><td>echenique, m; ...",http://dx.doi.org/10.1068/b39134,7,141,167,5,0.4246,2013,5,0.4246,0
1,8,ban (2013),NaN,"<table><tr><td>Authors:</td><td>ban, xg; ferri...",http://dx.doi.org/10.1016/j.trb.2012.11.003,2,221,506,12,1.0190,2013,12,1.0190,1
2,10,belis (2013),NaN,"<table><tr><td>Authors:</td><td>belis, d; shey...",http://dx.doi.org/10.1068/c11235,4,102,189,5,0.4246,2013,5,0.4246,2
3,12,clark (2013),NaN,"<table><tr><td>Authors:</td><td>clark, ta</td>...",http://dx.doi.org/10.1016/j.enpol.2012.11.006,3,239,407,7,0.5944,2013,7,0.5944,3
4,13,nie (2013),NaN,"<table><tr><td>Authors:</td><td>nie, y; shirmo...",http://dx.doi.org/10.3141/2333-04,2,128,199,11,0.9341,2013,11,0.9341,4


In [6]:
def get_topic(x):
    try:
        doi = x.replace('http://dx.doi.org/','')
        p = Doc.objects.get(
                wosarticle__di=doi,
                query=Query.objects.get(pk=qid),
                UT__icontains='WOS:'
            )
        topic = DocTopic.objects.filter(run_id=run_id,doc=p).order_by('-score').first().topic.title
    except:
        topic = ""
    return(topic)

In [7]:
df['main_topic'] = df.apply(lambda x: get_topic(x['url']),axis=1)

In [8]:
df.to_csv('../data/bibliometric_nodes.csv',index=False)

In [9]:
df

,Id,Label,timeset,description,url,cluster,weight<links>,weight<total_link_strength>,weight<citations>,weight<norm._citations>,score<pub._year>,score<citations>,score<norm._citations>,id,main_topic
0,1,echenique (2013),NaN,"<table><tr><td>Authors:</td><td>echenique, m; ...",http://dx.doi.org/10.1068/b39134,7,141,167,5,0.4246,2013,5,0.4246,0,Network optimisation
1,8,ban (2013),NaN,"<table><tr><td>Authors:</td><td>ban, xg; ferri...",http://dx.doi.org/10.1016/j.trb.2012.11.003,2,221,506,12,1.0190,2013,12,1.0190,1,Road tolls
2,10,belis (2013),NaN,"<table><tr><td>Authors:</td><td>belis, d; shey...",http://dx.doi.org/10.1068/c11235,4,102,189,5,0.4246,2013,5,0.4246,2,Climate change mitigation
3,12,clark (2013),NaN,"<table><tr><td>Authors:</td><td>clark, ta</td>...",http://dx.doi.org/10.1016/j.enpol.2012.11.006,3,239,407,7,0.5944,2013,7,0.5944,3,GHG emissions
4,13,nie (2013),NaN,"<table><tr><td>Authors:</td><td>nie, y; shirmo...",http://dx.doi.org/10.3141/2333-04,2,128,199,11,0.9341,2013,11,0.9341,4,Congestion charging
5,14,killip (2013),NaN,"<table><tr><td>Authors:</td><td>killip, g</td>...",http://dx.doi.org/10.1068/c11336,4,43,67,12,1.0190,2013,12,1.0190,5,Planning & governance
6,16,futcher (2013),NaN,"<table><tr><td>Authors:</td><td>futcher, ja; m...",http://dx.doi.org/10.1016/j.enpol.2012.10.080,1,69,74,11,0.9341,2013,11,0.9341,6,Urban form
7,19,aylett (2013),NaN,"<table><tr><td>Authors:</td><td>aylett, a</td>...",http://dx.doi.org/10.1068/c11304,4,84,121,17,1.4436,2013,17,1.4436,7,Planning & governance
8,27,burch (2013),NaN,"<table><tr><td>Authors:</td><td>burch, s; rayn...",http://dx.doi.org/10.1068/c1206,4,86,140,7,0.5944,2013,7,0.5944,8,Climate change mitigation
9,33,burch (2013),NaN,"<table><tr><td>Authors:</td><td>burch, s; rayn...",http://dx.doi.org/10.1068/c3105ed,4,58,89,10,0.8492,2013,10,0.8492,9,Planning & governance


In [10]:
Doc.objects.filter(wosarticle__di='10.1016/j.enbuild.2012.10.004',scopus=False)

<QuerySet [<Doc: WOS:000314378500008>]>

In [11]:
cluster_topics=[]
nodois=0
dts = DocTopic.objects.filter(run_id=run_id)
for index, row in df.iterrows():
    if row['url'] is not None:
        doi = str(row['url']).replace('http://dx.doi.org/','')
        try:
            p = Doc.objects.get(
                wosarticle__di=doi,
                query=Query.objects.get(pk=qid),
                UT__icontains='WOS:'
            )
        except:
            nodois+=1
            continue
        tops = dts.filter(doc=p)

        for t in tops:
            cluster_topics.append({
                "cluster": row['cluster'],
                "topic": t.topic.title,
                "score": t.score
            })

nodois

188

In [12]:
cluster_topics

[{'cluster': 7,
  'score': 0.00265452753699627,
  'topic': 'Planning & governance'},
 {'cluster': 7, 'score': 0.00221955669623769, 'topic': 'Active travel'},
 {'cluster': 7,
  'score': 0.00440290425719344,
  'topic': 'Building energy standards'},
 {'cluster': 7, 'score': 0.0442999231452246, 'topic': 'Urban form'},
 {'cluster': 7, 'score': 0.00263763135043907, 'topic': 'Public transport'},
 {'cluster': 7, 'score': 0.0571696534124543, 'topic': 'Network optimisation'},
 {'cluster': 7, 'score': 0.0217919923518826, 'topic': 'Travel behaviour'},
 {'cluster': 2, 'score': 0.0118747667379387, 'topic': 'Congestion charging'},
 {'cluster': 2,
  'score': 0.00831396505904932,
  'topic': 'Planning & governance'},
 {'cluster': 2, 'score': 0.0530800479881264, 'topic': 'Network optimisation'},
 {'cluster': 2, 'score': 0.00268201118706543, 'topic': 'Charging stations'},
 {'cluster': 2, 'score': 0.0581379284651005, 'topic': 'Road tolls'},
 {'cluster': 4,
  'score': 0.022929047349551,
  'topic': 'Building

In [13]:
cts = pd.DataFrame.from_dict(list(cluster_topics))

cts.head()

,cluster,score,topic
0,7,0.002655,Planning & governance
1,7,0.002220,Active travel
2,7,0.004403,Building energy standards
3,7,0.044300,Urban form
4,7,0.002638,Public transport


In [14]:
ct_sum = cts.groupby(['cluster', 'topic']).agg({'score': 'sum'})
# Change: groupby state_office and divide by sum
ct_pcts = ct_sum.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
ct_pcts

score
cluster topic                                       
1       Active travel                       8.009381
        Air quality                         1.983023
        Building energy standards          15.701686
        Building water use                  1.610419
        Bus rapid transit                   0.057075
        Car parking                         0.158896
        Charging stations                   0.117838
        Climate change mitigation           3.213499
        Congestion charging                 1.553077
        Cycling                             0.700279
        Energy efficiency & consumption    18.485804
        GHG emissions                       5.782599
        Heating & cooling systems           6.401046
        Household recycling                 1.168078
        Network optimisation                3.603557
        Planning & governance               7.019338
        Public transport                    0.668738
        Rail transit                        0.305233
        Road tolls                          0.013845
        System design                       3.329977
        Thermal comfort                    14.234389
        Transport services & provisioning   0.732858
        Travel behaviour                    0.575851
        Urban form                          1.570759
        Waste management                    0.307385
        Waste water                         0.504406
        e-Vehicles                          2.190964
2       Active travel                       1.664568
        Air quality                         1.227383
        Building energy standards           0.181105
...                                              ...
7       Building energy standards           0.314456
        Bus rapid transit                   1.798857
        Car parking                        40.072769
        Charging stations                   4.236388
        Climate change mitigation           0.864219
        Congestion charging                 8.644301
        Cycling                             0.842355
        Energy efficiency & consumption     0.250938
        GHG emissions                       0.389977
        Heating & cooling systems           0.083892
        Household recycling                 0.058108
        Network optimisation                7.045480
        Planning & governance               4.640722
        Public transport                    4.137425
        Rail transit                        2.526952
        Road tolls                          0.791602
        System design                       1.688460
        Thermal comfort                     0.419456
        Transport services & provisioning   1.948639
        Travel behaviour                    9.113274
        Urban form                          2.867071
        Waste management                    0.125473
        Waste water                         0.257280
        e-Vehicles                          1.630913
8       Active travel                       0.409604
        Air quality                         1.281825
        Planning & governance               3.734425
        System design                       1.540930
        Waste management                    2.472936
        Waste water                        90.560279

[194 rows x 1 columns]

In [18]:
ct_pcts = ct_pcts[ct_pcts['score']>5]
g = ct_pcts['score'].groupby(level=0, group_keys=False)

res = g.apply(lambda x: x.order(ascending=False))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


In [19]:
res

cluster  topic                            
1        Energy efficiency & consumption      18.485804
         Building energy standards            15.701686
         Thermal comfort                      14.234389
         Active travel                         8.009381
         Planning & governance                 7.019338
         Heating & cooling systems             6.401046
         GHG emissions                         5.782599
2        Congestion charging                  22.246536
         Road tolls                           19.570501
         Network optimisation                 12.889638
         Travel behaviour                      8.880208
         Charging stations                     7.559463
3        Urban form                           12.945389
         Planning & governance                12.333678
         Rail transit                         10.437850
         Public transport                     10.215562
         Travel behaviour                      7.890284
     

In [17]:
res.to_csv('../data/cluster_topics.csv')